# 1. Data Processing

In [1]:
from importlib.metadata import version

print("torch version:",version("torch"))
print("tiktoken version:",version("tiktoken"))


torch version: 2.5.1
tiktoken version: 0.9.0


## 1.1 Text Tokenization

In [2]:
# read an open source text dataset
with open("./the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()  
print(f"Total number of characters: {len(raw_text)}")
print(raw_text[:99])


Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re
# create a simple tokenizer that splits on both punctuations and white spaces.
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:38])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [4]:
print(f"Number of tokens: {len(preprocessed)}")

Number of tokens: 9235


In [5]:
print(f"Number of unique tokens: {len(set(preprocessed))}")


Number of unique tokens: 1133


In [6]:
sorted(set(preprocessed))

['',
 '\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 ';',
 '?',
 'A',
 'Ah',
 'Among',
 'And',
 'Are',
 'Arrt',
 'As',
 'At',
 'Be',
 'Begin',
 'Burlington',
 'But',
 'By',
 'Carlo',
 'Chicago',
 'Claude',
 'Come',
 'Croft',
 'Destroyed',
 'Devonshire',
 'Don',
 'Dubarry',
 'Emperors',
 'Florence',
 'For',
 'Gallery',
 'Gideon',
 'Gisburn',
 'Gisburns',
 'Grafton',
 'Greek',
 'Grindle',
 'Grindles',
 'HAD',
 'Had',
 'Hang',
 'Has',
 'He',
 'Her',
 'Hermia',
 'His',
 'How',
 'I',
 'If',
 'In',
 'It',
 'Jack',
 'Jove',
 'Just',
 'Lord',
 'Made',
 'Miss',
 'Money',
 'Monte',
 'Moon-dancers',
 'Mr',
 'Mrs',
 'My',
 'Never',
 'No',
 'Now',
 'Nutley',
 'Of',
 'Oh',
 'On',
 'Once',
 'Only',
 'Or',
 'Perhaps',
 'Poor',
 'Professional',
 'Renaissance',
 'Rickham',
 'Riviera',
 'Rome',
 'Russian',
 'Sevres',
 'She',
 'Stroud',
 'Strouds',
 'Suddenly',
 'That',
 'The',
 'Then',
 'There',
 'They',
 'This',
 'Those',
 'Though',
 'Thwing',
 'Thwings',
 'To',
 'Usually',
 'Veneti

##  1.2 Converting tokens into token IDs

In [16]:
# Create the vocabulary of our tokenizer
all_unique_words = sorted(list(set(preprocessed)))
vocab_size = len(all_unique_words)
print(vocab_size)

1133


In [17]:
all_unique_words.extend(["<|endoftext|>", "<|unk|>"])

In [18]:
# create the word to ID mapping
vocab = {token:integer for integer, token in enumerate(all_unique_words)}
for i, item in enumerate(vocab.items()):
   print(f"{item}")
   if i == 20:
       break


('', 0)
('\n', 1)
(' ', 2)
('!', 3)
('"', 4)
("'", 5)
('(', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)


In [19]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    # from text to token IDs
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # to handle unknown words/tokens
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    # from token IDs to text
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
        

In [20]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"Hello, It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[4, 1134, 8, 59, 5, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 4, 70, 10, 41, 854, 1111, 757, 796, 10]


In [11]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [21]:
tokenizer.decode(tokenizer.encode(text))


'" <|unk|>, It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 1.3 BytePair Encoding (BPE)

In [23]:
import tiktoken

In [24]:
tokenizer = tiktoken.get_encoding("gpt2")

In [25]:
# <|endoftext|> token that marks the end of a text and start of a new token.
# you insert it between each document/book/website.
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
# |<|endoftext|>| is not inlcuded in the vocab of gpt so we add it manually.
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [26]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [27]:
#  BPE can handle unknown words by breaking it down into sub-words.
tokenizer.encode("cvzxasdmfnsadfmnsa")

[33967, 42592, 292, 36020, 69, 5907, 324, 38353, 5907, 64]

In [28]:
tokenizer.decode([33967])

'cv'

## 1.4 Data Sampling with Sliding Window

In [29]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- LLM pre-training phase consider of predicting next token in the current sequence so the target/label should be shifted by always shifted by one.
- Sliding window technique bs a stride of the input length to prevent overlapping between the input sequences.
- If Input tokens are (hello, my, name, is, Mohamed, How, are, you, ?) and our sequence length is 4, so:

    s1 = (hello, my, name,is) - target1 = (my, name, is, Mohamed)
    
    s2 = (Mohamed, How, are, you) - target2 = (How, are, you, ?)

In [31]:
from dataloader import create_dataloader_v1

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=5, stride=5, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899,  2138],
        [  257,  7026, 15632,   438,  2016],
        [  257,   922,  5891,  1576,   438],
        [  568,   340,   373,   645,  1049],
        [ 5975,   284,   502,   284,  3285],
        [  326,    11,   287,   262,  6001],
        [  286,   465, 13476,    11,   339]])

Targets:
 tensor([[  367,  2885,  1464,  1807,  3619],
        [  402,   271, 10899,  2138,   257],
        [ 7026, 15632,   438,  2016,   257],
        [  922,  5891,  1576,   438,   568],
        [  340,   373,   645,  1049,  5975],
        [  284,   502,   284,  3285,   326],
        [   11,   287,   262,  6001,   286],
        [  465, 13476,    11,   339,   550]])


### Create Token Embeddings

In [32]:
import torch

In [33]:
inputs_ids = torch.tensor([2, 1, 4, 5])

In [45]:
output_dim = 256

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(tokenizer.n_vocab, output_dim)


In [46]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035,  ...,  1.3337,  0.0771, -0.0522],
        [ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [-1.3152, -0.0677, -0.1350,  ..., -0.3181, -1.3936,  0.5226],
        ...,
        [ 0.5871, -0.0572, -1.1628,  ..., -0.6887, -0.7364,  0.4479],
        [ 0.4438,  0.7411,  1.1263,  ...,  1.2091,  0.6781,  0.3331],
        [-0.2537,  0.1446,  0.7203,  ..., -0.2134,  0.2144,  0.3006]],
       requires_grad=True)


In [47]:
embedding_layer(torch.tensor([0]))

tensor([[ 3.3737e-01, -1.7778e-01, -3.0353e-01, -5.8801e-01,  3.4861e-01,
          6.6034e-01, -2.1964e-01, -3.7917e-01,  7.6711e-01, -1.1925e+00,
          6.9835e-01, -1.4097e+00,  1.7938e-01,  1.8951e+00,  4.9545e-01,
          2.6920e-01, -7.7020e-02, -1.0205e+00, -1.6896e-01,  9.1776e-01,
          1.5810e+00,  1.3010e+00,  1.2753e+00, -2.0095e-01,  4.9647e-01,
         -1.5723e+00,  9.6657e-01, -1.1481e+00, -1.1589e+00,  3.2547e-01,
         -6.3151e-01, -2.8400e+00, -1.3250e+00,  1.7843e-01, -2.1338e+00,
          1.0524e+00, -3.8848e-01, -9.3435e-01, -4.9914e-01, -1.0867e+00,
          8.8054e-01,  1.5542e+00,  6.2662e-01, -1.7549e-01,  9.8284e-02,
         -9.3507e-02,  2.6621e-01, -5.8504e-01,  8.7684e-01,  1.6221e+00,
         -1.4779e+00,  1.1331e+00, -1.2203e+00,  1.3139e+00,  1.0533e+00,
          1.3881e-01,  2.2473e+00, -8.0364e-01, -2.8084e-01,  7.6968e-01,
         -6.5956e-01, -7.9793e-01,  1.8383e-01,  2.2935e-01,  5.1463e-01,
          9.9376e-01, -2.5873e-01, -1.

In [49]:
token_embeddings = embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 5, 256])

### Positional Encoding

In [50]:
context_length = 5
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [51]:
pos_embeddings = pos_embedding_layer(torch.arange(5))
pos_embeddings.shape

torch.Size([5, 256])

In [53]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape

torch.Size([8, 5, 256])